###Unstructured Files

In [0]:
%fs ls dbfs:/FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/carga/,carga/,0,0
dbfs:/FileStore/tables/carga_jan/,carga_jan/,0,0
dbfs:/FileStore/tables/clientes_cartao.csv,clientes_cartao.csv,1000506,1640055124000
dbfs:/FileStore/tables/covid_basico.csv,covid_basico.csv,73827,1642017255000
dbfs:/FileStore/tables/imagens/,imagens/,0,0
dbfs:/FileStore/tables/json/,json/,0,0
dbfs:/FileStore/tables/online_retail/,online_retail/,0,0
dbfs:/FileStore/tables/online_retail.zip,online_retail.zip,14919036,1642302866000
dbfs:/FileStore/tables/parquet/,parquet/,0,0


In [0]:
raw_df = spark.read.format("image").load("dbfs:/FileStore/tables/imagens/")
raw_df.printSchema()
image_df = raw_df.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data")
image_df.write.option("header", "true").mode("overwrite").csv("/tmp/data-lake/images.csv")

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [0]:
from pyspark.sql.functions import max, lit
temp_df = image_df.withColumn("max_width", lit(image_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+--------------------+---------+
|              origin|height|width|nChannels|mode|                data|max_width|
+--------------------+------+-----+---------+----+--------------------+---------+
|dbfs:/FileStore/t...|   371| 1530|        3|  16|[FF FF FF FF FF F...|     1530|
+--------------------+------+-----+---------+----+--------------------+---------+



In [0]:
%fs ls dbfs:/tmp/data-lake/images.csv

path,name,size,modificationTime
dbfs:/tmp/data-lake/images.csv/_SUCCESS,_SUCCESS,0,1642559903000
dbfs:/tmp/data-lake/images.csv/_committed_4841227537587623814,_committed_4841227537587623814,459,1642559902000
dbfs:/tmp/data-lake/images.csv/_started_4841227537587623814,_started_4841227537587623814,0,1642559899000
dbfs:/tmp/data-lake/images.csv/part-00000-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-0-1-c000.csv,part-00000-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-0-1-c000.csv,178,1642559901000
dbfs:/tmp/data-lake/images.csv/part-00001-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-1-1-c000.csv,part-00001-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-1-1-c000.csv,179,1642559901000
dbfs:/tmp/data-lake/images.csv/part-00002-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-2-1-c000.csv,part-00002-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-2-1-c000.csv,178,1642559901000
dbfs:/tmp/data-lake/images.csv/part-00003-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-3-1-c000.csv,part-00003-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-3-1-c000.csv,178,1642559901000
dbfs:/tmp/data-lake/images.csv/part-00004-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-4-1-c000.csv,part-00004-tid-4841227537587623814-7b99bb57-74d8-4d85-8447-5cfd2671a6db-4-1-c000.csv,109,1642559900000


###Semi-Structured

In [0]:
csv_df = spark.read.options(header="true", inferSchema="true").csv("/tmp/data-lake/images.csv")
csv_df.printSchema()
csv_df.show()

root
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- nChannels: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- data: string (nullable = true)

+--------------------+------+-----+---------+----+-----------+
|              origin|height|width|nChannels|mode|       data|
+--------------------+------+-----+---------+----+-----------+
|dbfs:/FileStore/t...|   371| 1530|        3|  16|[B@752b44f1|
|dbfs:/FileStore/t...|   268|  763|        3|  16|[B@2795e30c|
|dbfs:/FileStore/t...|   279| 1480|        3|  16| [B@4f711f4|
|dbfs:/FileStore/t...|   323| 1504|        3|  16| [B@b94ee9f|
|dbfs:/FileStore/t...|   197|  699|        3|  16|[B@23186a9d|
|dbfs:/FileStore/t...|   244|  617|        3|  16|[B@6ed47221|
|dbfs:/FileStore/t...|   224|  765|        3|  16|[B@711d23fd|
|dbfs:/FileStore/t...|   250|  766|        3|  16|[B@2a4c1221|
|dbfs:/FileStore/t...|   124|  456|        3|  16|[B@452b113a|
+-------

In [0]:
from pyspark.sql.functions import max, lit
temp_df = csv_df.withColumn("max_width", lit(csv_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+-----------+---------+
|              origin|height|width|nChannels|mode|       data|max_width|
+--------------------+------+-----+---------+----+-----------+---------+
|dbfs:/FileStore/t...|   371| 1530|        3|  16|[B@752b44f1|     1530|
+--------------------+------+-----+---------+----+-----------+---------+



###Structured Data

In [0]:
image_df.write.parquet("/tmp/data-lake/images.parquet")
parquet_df = spark.read.parquet("/tmp/data-lake/images.parquet")
parquet_df.printSchema()
display(parquet_df)

root
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- nChannels: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- data: binary (nullable = true)



origin,height,width,nChannels,mode,data
dbfs:/FileStore/tables/imagens/DSC_0105.jpg,279,1480,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0106.jpg,323,1504,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0104.jpg,371,1530,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0108.jpg,268,763,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0107.jpg,224,765,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0100.jpg,250,766,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0109.jpg,197,699,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0101.jpg,244,617,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/tables/imagens/DSC_0110.jpg,124,456,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.


In [0]:
from pyspark.sql.functions import max, lit
temp_df = parquet_df.withColumn("max_width", lit(parquet_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+--------------------+---------+
|              origin|height|width|nChannels|mode|                data|max_width|
+--------------------+------+-----+---------+----+--------------------+---------+
|dbfs:/FileStore/t...|   371| 1530|        3|  16|[FF FF FF FF FF F...|     1530|
+--------------------+------+-----+---------+----+--------------------+---------+



In [0]:
%fs ls /tmp/data-lake/images.parquet

path,name,size
dbfs:/tmp/data-lake/images.parquet/_SUCCESS,_SUCCESS,0
dbfs:/tmp/data-lake/images.parquet/_committed_3764051096931124086,_committed_3764051096931124086,524
dbfs:/tmp/data-lake/images.parquet/_started_3764051096931124086,_started_3764051096931124086,0
dbfs:/tmp/data-lake/images.parquet/part-00000-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-176-1-c000.snappy.parquet,part-00000-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-176-1-c000.snappy.parquet,1101459
dbfs:/tmp/data-lake/images.parquet/part-00001-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-177-1-c000.snappy.parquet,part-00001-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-177-1-c000.snappy.parquet,1228108
dbfs:/tmp/data-lake/images.parquet/part-00002-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-178-1-c000.snappy.parquet,part-00002-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-178-1-c000.snappy.parquet,1042820
dbfs:/tmp/data-lake/images.parquet/part-00003-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-179-1-c000.snappy.parquet,part-00003-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-179-1-c000.snappy.parquet,290031
dbfs:/tmp/data-lake/images.parquet/part-00004-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-180-1-c000.snappy.parquet,part-00004-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-180-1-c000.snappy.parquet,307002


In [0]:
%fs rm -r /tmp/data-lake/images.parquet

res4: Boolean = true